In [ ]:
#https://www.kaggle.com/code/charel/learn-by-example-rnn-lstm-gru-time-series/notebook
#https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
#https://curiousily.com/posts/time-series-forecasting-with-lstm-for-daily-coronavirus-cases/

#https://jtd.amegroups.com/article/view/36385/html
#https://github.com/curiousily/How-to-Predict-Bitcoin-Prices-using-LSTMs-in-Keras/blob/master/predicting_stock_prices.ipynb
#https://www.kaggle.com/code/nicapotato/keras-timeseries-multi-step-multi-output/notebook
    
    
#LSTM Model combine climate factors

import numpy as np # linear algebra
from numpy import newaxis
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import os

print ('import completed')


In [ ]:
sc = MinMaxScaler(feature_range=(0,1))

dt=pd.read_csv("C:/Users/sally/Data/Peru_Climate.csv")
dt=dt[['Date','Year', 'EW','temp','humidity','precip']]


In [ ]:

dfC = pd.read_csv("C:/Users/sally/Data/COVID-19-Peru.csv")

dt=pd.read_csv("C:/Users/sally/Data/Peru_Climate.csv")
dt=dt[['Date','Year', 'EW','temp','humidity','precip']]

dfC = pd.merge(dfC,dt, on=['Year','EW','Date'])

dfC['Week']=(dfC['Year']-2020)*52+dfC['EW']

dfC.set_index('Date')

dfC['New_cases'] = dfC['New_cases'].rolling(7).mean()
dfC['New_cases'].fillna(0, inplace=True)

dfC=dfC[['Year', 'EW','New_cases','temp','humidity','precip']]
dfC=dfC.interpolate()


print(dfC)



In [ ]:

df=dfC[['temp','humidity','precip','New_cases']]

df[['temp','humidity','precip','New_cases']]=sc.fit_transform(df)
print(df)


In [ ]:
def load_data_all(datasetname, seq_len, rt):
    # A support function to help prepare datasets for an RNN/LSTM/GRU
    data = datasetname.loc[:,]
        
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])   

    result = np.array(result)

    row = round(rt * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))  
    return [x_train, y_train, x_test, y_test]


feature_train, label_train, feature_test, label_test = load_data_all(df, 21, 0.8)

print(feature_train.shape)
print(feature_test.shape)
print(label_train.shape)
print(label_test.shape)

In [ ]:
print(feature_train[0])
print(label_train[0])

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(feature_train.shape[1], feature_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "linear"))

model.compile(loss='mse', optimizer='adam')

print ('model compiled')

print (model.summary())

In [ ]:
history=model.fit(feature_train, label_train[:,-1], batch_size=512, epochs=200, validation_data = (feature_test, label_test[:,-1]))

In [ ]:
plt.plot(history.history['loss'], color='blue')
plt.plot(history.history['val_loss'], color='red')
plt.legend(['Loss','Validation Loss'], prop={'size': 10}, loc='upper center', bbox_to_anchor=(0.5, 1.02), ncol=3, fancybox=True, shadow=True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.savefig('C:/Users/sally/Data/Peru_LSTM_COVID_Climate_Loss.png')
plt.show()
plt.close()

In [ ]:
predicted_test = model.predict(feature_test)

def plot_results_1(predicted_data, true_data): 
    fig = plt.figure(facecolor='white') 
    ax = fig.add_subplot(111) 
    ax.plot(true_data, label='True Data') 
    plt.plot(predicted_data, label='Prediction') 
    plt.legend() 
    plt.show() 

plot_results_1(predicted_test, label_test[:,-1])

In [ ]:
#####END#####

In [ ]:
workdir = "C:/Users/sally/Data/"
def plot_results_1(predicted_data, true_data): 
    fig = plt.figure(facecolor='white') 
    ax = fig.add_subplot(111) 
    ax.plot(true_data, label='True Data') 
    plt.plot(predicted_data, label='Prediction') 
    plt.legend() 
    plt.show() 
    #filename = str(figname) + '.png'
    #plt.savefig(os.path.join(workdir, filename))

def plot_results_2(predicted_data, true_data, figname): 
    fig = plt.figure(facecolor='white') 
    ax = fig.add_subplot(111) 
    ax.plot(true_data, label='True Data') 
    plt.plot(predicted_data, label='Prediction') 
    plt.legend() 
    plt.show() 
    filename = str(figname) + '.png'
    fig.savefig(os.path.join(workdir, filename)) 
    
predicted_test = model.predict(feature_test)

print(predicted_test)
print(label_test[:,-2])


In [ ]:
plot_results_1(predicted_test,label_test[:,-2])

In [ ]:
model2 = Sequential()
model2.add(LSTM(50, return_sequences=True, input_shape=(feature_train.shape[1], feature_train.shape[2])))
model2.add(Dropout(0.2))
model2.add(LSTM(100, return_sequences=False))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation = "linear"))

model2.compile(loss='mse', optimizer='adam')

print ('model compiled')

print (model2.summary())

In [ ]:
model2.fit(feature_train, label_train[:,-1], batch_size=512, epochs=50, validation_data = (feature_test, label_test[:,-1]))

In [ ]:
predicted_test2 = model2.predict(feature_test)

print(predicted_test2)
print(label_test[:,-1])

In [ ]:
plot_results_1(predicted_test2,label_test[:,-1])

In [ ]:

def predict_future_two(feature_test, model1, model2, period):
    future=[]
    test=feature_test[-1:].copy()
    sp=test.shape[1]

    for _ in range(period):    
        test2=test.copy()     
        p1=model.predict(test)[0][0]   
        p2=model2.predict(test)[0][0]   
        future.append([p1,p2])
        for i in range(sp-1):
            test[0][i][0]=test2[0][i+1][0]
            test[0][i][1]=test2[0][i+1][1]
        test[0][sp-1][0]=future[-1][0]
        test[0][sp-1][1]=future[-1][1]
    return future

future=predict_future_two(feature_test, model, model2, 100)


In [ ]:
future2 = np.array(future)
plt.plot(future2[:,0])
plt.plot(future2[:,1])


In [ ]:
print(predicted_test2)

In [ ]:
predict=[[predicted_test[i][0], predicted_test2[i][0] ] for i in range(len(predicted_test))]
predict2 = np.array(predict)

future3 = future2.reshape(future2.shape[0], 2)
predict3 = predict2.reshape(predict2.shape[0], 2)

predict3 = np.append(predict3,future3,axis=0 )

predict3 = sc.inverse_transform(predict3)

label_test2 = sc.inverse_transform(label_test.reshape(label_test.shape[0],2))

plot_results_2(predict3[:,0],label_test2[:,0], "Peru_Dengue_comb_pred_day")
plot_results_2(predict3[:,1],label_test2[:,1], "Peru_COVID_comb_pred_day")


In [ ]:
predict4=np.log10(predict3+1)
label_test3=np.log10(label_test2+1)

plot_results_2(predict4[:,0],label_test3[:,0], "Peru_Dengue_log_comb_pred_day")
plot_results_2(predict4[:,1],label_test3[:,1], "Peru_COVID_log_comb_pred_day")


In [ ]:
def predict_future_two_start_climate(feature_test, model1, model2, period):
    future=[]
    test=feature_test.copy()
    sp=test.shape[1]

    for i in range(period-1):   
         
        p1=model.predict(test[i:i+1])[0][0]   
        p2=model2.predict(test[i:i+1])[0][0]   
        future.append([p1,p2])
        for i in range(sp-1):
            test[0][i][0]=test2[0][i+1][0]
            test[0][i][1]=test2[0][i+1][1]
        test[0][sp-1][0]=future[-1][0]
        test[0][sp-1][1]=future[-1][1]
    return future

future_start=predict_future_two_start(feature_test, model, model2, feature_test.shape[0])
future2_start = np.array(future_start)
plt.plot(future2_start[:,0])
plt.plot(future2_start[:,1])

In [ ]:
predict3_start = sc.inverse_transform(future2_start)

In [ ]:
plot_results_2(predict3_start[:,0],label_test2[:,0], "Peru_Dengue_comb_pred_day_start")
plot_results_2(predict3_start[:,1],label_test2[:,1], "Peru_COVID_comb_pred_day_start")

In [ ]:
print(feature_test.shape)

In [ ]:
def predict_future_two_batch(feature_test, model1, model2, period):
    future=[]   
    lx = feature_test.shape[0]
    sp = feature_test.shape[1]    
    n = lx//period+1
    for batch in range(n):
        test=feature_test[batch*period:batch*period+1].copy()
        for _ in range(period):
        
        
            test2=test.copy()     
            p1=model.predict(test)[0][0]   
            p2=model2.predict(test)[0][0]   
            future.append([p1,p2])
            for i in range(sp-1):
                test[0][i][0]=test2[0][i+1][0]
                test[0][i][1]=test2[0][i+1][1]
            test[0][sp-1][0]=future[-1][0]
            test[0][sp-1][1]=future[-1][1]
    return future

future_batch=predict_future_two_batch(feature_test, model, model2, 21)
future2_batch = np.array(future_batch)
plt.plot(future2_batch[:,0])
plt.plot(future2_batch[:,1])

In [ ]:
predict3_batch = sc.inverse_transform(future2_batch)
plot_results_2(predict3_batch[:,0],label_test2[:,0], "Peru_Dengue_comb_pred_day_batch")
plot_results_2(predict3_batch[:,1],label_test2[:,1], "Peru_COVID_comb_pred_day_batch")